In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [55]:
df = pd.read_csv('data/clean_df.csv')

In [56]:
df.head()

,date,incident_id,governorate,district,area,target,main_category,sub_category,min_air_raids,max_air_raids,civilian_casualties,fatalities,woman_fatalities,child_fatalities,injured,woman_injured,child_injured,confirmed_time,time_of_day
0,2015-03-26,1,Sanaa,bani al-harith,Al-Rahabah,Al-Daylami Airbase,military_security_target,military site,1,2.0,0,0,0,0,0,0,0.0,2,morning
1,2015-03-26,2,Sanaa,bani al-harith,Airport,Sana'a International Airport,infrastructure,transport,1,2.0,0,0,0,0,0,0,0.0,2,morning
2,2015-03-26,3,Sanaa,bani al-harith,Al-Sonblah Neighbourhood,Residential Area,civilian,residential area,1,2.0,29,21,3,14,8,0,7.0,2,morning
3,2015-03-26,4,Sanaa,al-sab'ein,Al-Nahdain,Presidential Palace,military_security_target,military site,1,2.0,0,0,0,0,0,0,0.0,3,morning
4,2015-03-26,5,Sanaa,al-thawrah,Al-Nahdhah,Former 1st Armoured Division,military_security_target,military site,1,2.0,0,0,0,0,0,0,0.0,3,morning


In [66]:
df['date'] = pd.to_datetime(df.date)
df.index = df['date']
df.resample('D').sum()

# df.set_index('date', inplace=True)
# df.index = pd.to_datetime(df.index)
#df.index = pd.DatetimeIndex(df.index).to_period('D')

,incident_id,min_air_raids,max_air_raids,civilian_casualties,fatalities,woman_fatalities,child_fatalities,injured,woman_injured,child_injured
date,,,,,,,,,,
2015-01-04,6758,31,63.0,2,2,2,0,0,0,0.0
2015-01-05,29257,34,261.0,197,83,8,5,114,14,9.0
2015-01-06,46224,32,72.0,10,5,0,0,5,0,0.0
2015-01-07,71295,35,102.0,11,8,2,1,3,0,2.0
2015-01-08,26255,10,60.0,11,8,3,2,3,1,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-12-27,202131,9,12.0,0,0,0,0,0,0,0.0
2020-12-28,157269,7,14.0,0,0,0,0,0,0,0.0
2020-12-29,157318,7,11.0,0,0,0,0,0,0,0.0


In [58]:
y_train, y_test = train_test_split(df['civilian_casualties'], random_state=6, shuffle=False)

In [59]:
df.index.is_monotonic

True

In [60]:
model = ARIMA(endog=y_train, order=(1, 0, 1))

In [61]:
df['civilian_casualties'].index

RangeIndex(start=0, stop=22485, step=1)